In [1]:
import array
import random
import numpy as np
from deap import algorithms, base, creator, tools
from scoop import futures

In [2]:
def fitness_function(chromossome):
    return sum(chromossome),

In [3]:
# Cria o tipo de função fitness e o indivíduo
creator.create('FitnessMax', base.Fitness, weights=(1, ))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
# Registra os nomes e tipos de indivíduo, fitness e população
toolbox.register('attr_bool', random.randint, 0, 1)
toolbox.register('individual', tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, 300)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

# Registra os operadores
toolbox.register('evaluate', fitness_function)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=.01)
toolbox.register('select', tools.selTournament, tournsize=5)

In [4]:
# Registra a execução em paralelismo
toolbox.register('map', futures.map)

In [5]:
# Parâmetros do algirtmo
CXPB = .8
MUTPB = .1
NGEN = 400
FREQ = 10
NPOP = 30
NISLANDS = 5


In [6]:
# Execução do GA em ilhas
islands = [toolbox.population(n=NPOP) for _ in range(NISLANDS)]
toolbox.register('algorithm', algorithms.eaSimple, toolbox=toolbox, cxpb=CXPB,
                 mutpb=MUTPB, ngen=FREQ, verbose=False)

for _ in range(0, NGEN, FREQ):
    results = toolbox.map(toolbox.algorithm, islands)
    islands = [pop for pop, logbook in results]
    tools.migRing(islands, 5, tools.selBest)

e:\workspace\pesquisa-operacional\Island_model_DEAP\.venv\lib\site-packages\scoop\fallbacks.py:38: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().
  warnings.warn(


In [7]:
# Imprime o fitness máximo e médio por ilhas
for pop in islands:
    # Armazena o fitness final de cada ilha
    final_fitness = []
    for individual in pop:
        final_fitness.append(individual.fitness.values)
    print(f'Max fitness: {max(final_fitness)[0]}; Avg fitness: {np.mean(final_fitness)}')

Max fitness: 300.0; Avg fitness: 299.96666666666664
Max fitness: 300.0; Avg fitness: 299.7
Max fitness: 300.0; Avg fitness: 299.8
Max fitness: 300.0; Avg fitness: 299.8666666666667
Max fitness: 300.0; Avg fitness: 299.43333333333334
